In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
from sklearn.model_selection import train_test_split

# Load datasets

X_train = pd.read_parquet(
    r"https://github.com/monatagelsir7/enivornmental_impact_of_aviation/raw/refs/heads/main/Train-test-validation_with_outliers/X_train.parquet"
)
X_test = pd.read_parquet(
    r"https://github.com/monatagelsir7/enivornmental_impact_of_aviation/raw/refs/heads/main/Train-test-validation_with_outliers/X_test.parquet"
)
X_val = pd.read_parquet(
    r"https://github.com/monatagelsir7/enivornmental_impact_of_aviation/raw/refs/heads/main/Train-test-validation_with_outliers/X_val.parquet"
)
y_train = pd.read_parquet(
    r"https://github.com/monatagelsir7/enivornmental_impact_of_aviation/raw/refs/heads/main/Train-test-validation_with_outliers/y_train.parquet"
).squeeze()
y_test = pd.read_parquet(
    r"https://github.com/monatagelsir7/enivornmental_impact_of_aviation/raw/refs/heads/main/Train-test-validation_with_outliers/y_test.parquet"
).squeeze()
y_val = pd.read_parquet(
    r"https://github.com/monatagelsir7/enivornmental_impact_of_aviation/raw/refs/heads/main/Train-test-validation_with_outliers/y_val.parquet"
).squeeze()

In [2]:
df = pd.read_parquet(
    r"https://github.com/monatagelsir7/enivornmental_impact_of_aviation/raw/refs/heads/main/cleaned_aviation_data_with_outliers_v4.parquet"
)
df

,airline_iata,acft_icao,acft_class,seymour_proxy,source,seats,n_flights,iata_departure,iata_arrival,departure_lon,...,distance_km,ask,rpk,fuel_burn_seymour,fuel_burn,co2,domestic,log_co2,log_distance,co2_per_distance
0,1SQ,PA32,PP,P28B,BTS,62112.0,11466.5,SPN,TIQ,145.729004,...,17.765225,1.103434e+06,9.092294e+05,20.486379,2.349071e+05,7.423063e+05,1,13.517519,2.932005,41784.233355
1,1SQ,PA32,PP,P28B,BTS,62112.0,11466.5,TIQ,SPN,145.619003,...,17.765225,1.103434e+06,9.092294e+05,20.486379,2.349071e+05,7.423063e+05,1,13.517519,2.932005,41784.233355
2,HA,B712,NB,B712,BTS,1246720.0,9740.0,HNL,OGG,-157.924228,...,162.001417,2.019704e+08,1.664236e+08,1305.895783,1.271942e+07,4.019338e+07,1,17.509213,5.093759,248105.130283
3,HA,B712,NB,B712,BTS,1246720.0,9740.0,OGG,HNL,-156.431212,...,162.001417,2.019704e+08,1.664236e+08,1305.895783,1.271942e+07,4.019338e+07,1,17.509213,5.093759,248105.130283
4,HA,B712,NB,B712,BTS,879296.0,6869.5,HNL,KOA,-157.924228,...,262.778946,2.310605e+08,1.903938e+08,1567.819046,1.077013e+07,3.403362e+07,1,17.342859,5.575111,129514.257569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291639,ACN,None,None,None,ANAC,4.5,4.0,SXO,CGB,-50.689602,...,736.539053,3.314426e+03,2.731087e+03,4223.750606,1.010269e+02,3.192451e+02,1,5.769087,6.603319,0.433439
291640,ACN,None,None,None,ANAC,4.5,4.0,QDV,CGH,-46.943921,...,57.645994,2.594070e+02,2.137513e+02,4223.750606,4.807282e+01,1.519101e+02,1,5.029850,4.071519,2.635224
291643,ACN,None,None,None,ANAC,4.5,4.0,QDV,POO,-46.943921,...,153.120924,6.890442e+02,5.677724e+02,4223.750606,5.527809e+01,1.746788e+02,1,5.168657,5.037738,1.140790
291644,ACN,None,None,None,ANAC,4.5,4.0,RRJ,QDV,-43.372194,...,366.577348,1.649598e+03,1.359269e+03,4223.750606,7.167492e+01,2.264927e+02,1,5.427118,5.906934,0.617858


In [3]:
# X_train["domestic"] = X_train["domestic_x"]  # or "domestic_y"
# X_train.drop(columns=["domestic_x", "domestic_y"], inplace=True)

# X_train["domestic"] = X_train["domestic"].astype("category")

# X_val["domestic"] = X_val["domestic_x"]
# X_val.drop(columns=["domestic_x", "domestic_y"], inplace=True)
# X_val["domestic"] = X_val["domestic"].astype("category")

# X_test["domestic"] = X_test["domestic_x"]
# X_test.drop(columns=["domestic_x", "domestic_y"], inplace=True)
# X_test["domestic"] = X_test["domestic"].astype("category")

In [4]:
for i in [X_train, X_test, X_val]:
    i["is_widebody"] = i["is_widebody"].astype("category")
    i["domestic"] = i["domestic"].astype("category")
    i["is_narrowbody"] = i["is_narrowbody"].astype("category")

In [5]:
original_cats = df[
    [
        "acft_class",
        "departure_country",
        "arrival_country",
        "departure_continent",
        "arrival_continent",
        # "domestic",
        # "is_widebody",
        # "is_narrowbody",
    ]
]

In [6]:
# Align index to encoded splits
X_train = X_train.merge(original_cats, left_index=True, right_index=True)
X_val = X_val.merge(original_cats, left_index=True, right_index=True)
X_test = X_test.merge(original_cats, left_index=True, right_index=True)

In [7]:
numeric_features = X_train.select_dtypes(include=["int64", "float64"]).columns.tolist()
numeric_features

['seats',
 'n_flights',
 'ask',
 'rpk',
 'fuel_burn',
 'fuel_burn_per_seat',
 'ask_per_seat',
 'ask_per_flight']

In [8]:
original_cats = [
    "acft_class",
    "departure_country",
    "arrival_country",
    "departure_continent",
    "arrival_continent",
    "domestic",
    "is_widebody",
    "is_narrowbody",
    "same_continent",
]

In [9]:
encoded_only_cols = [
    col
    for col in X_train.columns
    if col not in original_cats and col not in numeric_features
]
X_train.drop(columns=encoded_only_cols, inplace=True)
X_test.drop(columns=encoded_only_cols, inplace=True)
X_val.drop(columns=encoded_only_cols, inplace=True)

In [10]:
X_train

,seats,n_flights,domestic,ask,rpk,fuel_burn,same_continent,fuel_burn_per_seat,ask_per_seat,ask_per_flight,is_widebody,is_narrowbody,acft_class,departure_country,arrival_country,departure_continent,arrival_continent
58430,10465.000000,57.5,1,2.938172e+07,2.421054e+07,4.652493e+05,True,44.457651,2807.618186,5.109865e+05,0,1,NB,US,US,NA,NA
14544,2896.000000,362.0,1,1.694946e+06,1.396635e+06,8.759352e+04,True,30.246381,585.271316,4.682171e+03,0,0,TP,US,US,NA,NA
1846,84480.714650,1204.5,1,1.655428e+07,1.364072e+07,5.234898e+05,True,6.196560,195.953331,1.374369e+04,0,0,TP,DK,DK,EU,EU
54864,4734.286624,67.5,0,2.427359e+06,2.000144e+06,5.740993e+04,False,12.126416,512.719083,3.596088e+04,0,0,TP,EH,MA,AF,AF
167771,249.093750,1.5,0,3.495171e+05,2.880021e+05,7.456522e+03,False,29.934600,1403.154844,2.330114e+05,0,1,NB,MA,FR,AF,EU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279080,7537.253500,4.0,0,1.093306e+07,9.008846e+06,2.695980e+05,False,35.768734,1450.536981,2.733266e+06,0,0,None,IN,MM,AS,AS
132911,586.404777,3.0,0,1.724257e+05,1.420788e+05,6.839698e+03,False,11.663782,294.038697,5.747523e+04,0,1,NB,CH,DE,EU,EU
198702,14.000000,1.0,0,1.100371e+05,9.067059e+04,1.935812e+04,False,1382.722798,7859.794836,1.100371e+05,0,0,PJ,US,CH,NA,EU
30457,13169.761510,174.0,0,1.078796e+07,8.889277e+06,2.967378e+05,False,22.531755,819.145991,6.199975e+04,0,0,TP,AT,CH,EU,EU


In [11]:
for df_split in [X_train, X_val, X_test]:
    df_split.drop(columns="seats", inplace=True)

In [12]:
for i in X_train, X_test, X_val:
    i["acft_class"].fillna("Unknown", inplace=True)

/var/folders/cv/pb7bdsd97cq7qt_b65khbfvr0000gn/T/ipykernel_78815/1062177129.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  i["acft_class"].fillna("Unknown", inplace=True)


In [13]:
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

categorical_cols = [
    "acft_class",
    "departure_country",
    "arrival_country",
    "departure_continent",
    "arrival_continent",
    "domestic",
    "is_widebody",
    "is_narrowbody",
    "same_continent",
    # "same_country",
]

categorical_cols = [
    "acft_class",
    "departure_country",
    "arrival_country",
    "departure_continent",
    "arrival_continent",
    "domestic",
    "is_widebody",
    "is_narrowbody",
    "same_continent",
]

for col in categorical_cols:
    X_train[col] = X_train[col].astype("category")
    X_test[col] = X_test[col].astype("category")
    X_val[col] = X_val[col].astype("category")

train_pool = Pool(X_train, y_train, cat_features=categorical_cols)
val_pool = Pool(X_val, y_val, cat_features=categorical_cols)
test_pool = Pool(X_test, y_test, cat_features=categorical_cols)

model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.1,
    depth=8,
    loss_function="RMSE",
    early_stopping_rounds=50,
    verbose=100,
)

model.fit(train_pool, eval_set=val_pool)

y_val_pred = model.predict(X_val)
y_test_pred = model.predict(X_test)

print("\nCatboost Validation Set Evaluation:")
print("R²:", r2_score(y_val, y_val_pred))
print("MAE:", mean_absolute_error(y_val, y_val_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_val, y_val_pred)))

print("\n Catboost Test Set Evaluation:")
print("R²:", r2_score(y_test, y_test_pred))
print("MAE:", mean_absolute_error(y_test, y_test_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_test_pred)))

0:	learn: 5123.9242731	test: 4953.3569385	best: 4953.3569385 (0)	total: 110ms	remaining: 1m 50s
100:	learn: 295.1735379	test: 366.3034545	best: 366.3034545 (100)	total: 5.21s	remaining: 46.3s
200:	learn: 217.6004664	test: 303.9464536	best: 303.9464536 (200)	total: 11s	remaining: 43.9s
300:	learn: 181.5564850	test: 277.2179326	best: 277.2179326 (300)	total: 16.2s	remaining: 37.7s
400:	learn: 159.3778027	test: 261.8106130	best: 261.8106130 (400)	total: 21.8s	remaining: 32.5s
500:	learn: 144.7615016	test: 251.7446504	best: 251.7446504 (500)	total: 27s	remaining: 26.9s
600:	learn: 133.5191629	test: 245.1262715	best: 245.1262715 (600)	total: 32.5s	remaining: 21.6s
700:	learn: 124.3415398	test: 240.7049229	best: 240.7049229 (700)	total: 37.9s	remaining: 16.1s
800:	learn: 117.0396553	test: 236.6568525	best: 236.6568525 (800)	total: 43.1s	remaining: 10.7s
900:	learn: 111.1402364	test: 233.8380001	best: 233.8380001 (900)	total: 48.7s	remaining: 5.35s
999:	learn: 106.0314936	test: 231.0712694	be

In [14]:
# import shap

# shap.initjs()

# # Last column is expected value)
# shap_values = model.get_feature_importance(
#     Pool(X_test, label=y_test, cat_features=categorical_cols),
#     type="ShapValues"
# )

# expected_value = shap_values[0, -1]         # same for all rows
# shap_values = shap_values[:, :-1]           # remove last column

# shap.force_plot(
#     base_value=expected_value,
#     shap_values=shap_values[3],
#     features=X_test.iloc[3],
#     feature_names=X_test.columns
# )

In [15]:
# feature = "acft_class"
# model.get_feature_statistics(X_train, y_train, feature, plot=True)

### Predictions and fitting on whole train dataste

In [16]:
X_full_train = pd.concat([X_train, X_val])
y_full_train = pd.concat([y_train, y_val])

In [17]:
train_pool = Pool(X_full_train, y_full_train, cat_features=categorical_cols)
test_pool = Pool(X_test, y_test, cat_features=categorical_cols)

# Initialize and fit the final model
final_model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.1,
    depth=8,
    loss_function="RMSE",
    early_stopping_rounds=50,
    verbose=100,
)

final_model.fit(train_pool, eval_set=test_pool)

y_test_pred = final_model.predict(X_test)
y_all_pred = final_model.predict(pd.concat([X_train, X_val, X_test]))

# Evaluate
print("Final Model Evaluation on Test Set:")
print("R²:", r2_score(y_test, y_test_pred))
print("MAE:", mean_absolute_error(y_test, y_test_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_test_pred)))

0:	learn: 5095.9784108	test: 5150.9407065	best: 5150.9407065 (0)	total: 80.7ms	remaining: 1m 20s
100:	learn: 289.3488017	test: 356.9841594	best: 356.9841594 (100)	total: 6.27s	remaining: 55.8s
200:	learn: 216.9069000	test: 284.4649766	best: 284.4649766 (200)	total: 12.9s	remaining: 51.2s
300:	learn: 182.5759227	test: 258.8011614	best: 258.8011614 (300)	total: 19.9s	remaining: 46.2s
400:	learn: 161.3101286	test: 244.2524431	best: 244.2524431 (400)	total: 26.9s	remaining: 40.1s
500:	learn: 146.2437857	test: 234.6528674	best: 234.6528674 (500)	total: 33.8s	remaining: 33.7s
600:	learn: 134.3396921	test: 227.5508369	best: 227.5508369 (600)	total: 40.5s	remaining: 26.9s
700:	learn: 125.5480734	test: 222.6376913	best: 222.6376913 (700)	total: 47.5s	remaining: 20.3s
800:	learn: 117.8219006	test: 218.6238067	best: 218.5934270 (799)	total: 54.4s	remaining: 13.5s
900:	learn: 112.0452580	test: 215.2726528	best: 215.2726528 (900)	total: 1m 1s	remaining: 6.71s
999:	learn: 107.1480754	test: 212.95304

In [18]:
pd.DataFrame({"y_pred": y_all_pred}).to_csv(
    "catboost_final_predictions.csv", index=False
)